In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, ".");

In [ ]:
using Images
using VideoIO: openvideo
using ImageView: imshow
using StaticArrays: SVector, SMatrix
using CoordinateTransformations
using Base.Test
using BenchmarkTools
using Interact
using Interpolations
using Unitful

usable motion from 3s to 20s

In [ ]:
import Cornercam
reload("Cornercam")

In [ ]:
target_size = (30, 50)

In [ ]:
video = VideoIO.openvideo("DSCF7139.MOV")

In [ ]:
background = Cornercam.mean_frame(video, (10u"s", 12u"s"))
save("background.png", background)
background

In [ ]:
original_corners = SVector{2, Int}[[236, 404], 
    [207, 1188], 
    [694, 1276], 
    [696, 340]]

desired_corners = SVector{2, Int}[[1, 1], 
    [0, target_size[2]-1],
    [target_size[1]-1, target_size[2]-1],
    [target_size[1]-1, 0]]

In [ ]:
H = Cornercam.rectify(desired_corners, original_corners)
@test all((H.(desired_corners) .≈ original_corners))

In [ ]:
samples = Cornercam.polar_samples(
    linspace(0, π/2, 200),
    linspace(0.01, 30, 50)
);

θs = linspace(0, π/2, 200)
σ = 0.085
params = Cornercam.Params(θs, samples, σ)
cam = Cornercam.CornerCamera(video, H, background, params)

In [ ]:
Cornercam.show_samples(cam)

In [ ]:
A = Cornercam.visibility_gain(samples, θs);

In [ ]:
λ = sqrt(2.7)
gain = Cornercam.cornercam_gain(A, σ, λ);

In [ ]:
function sample(im, samples, H)
    itp = interpolate(imfilter(im, Kernel.gaussian(5)), BSpline(Linear()), OnGrid())
    [itp[Tuple(H(s))...] for s in samples]
end

In [ ]:
function imnormal(im)
    lb = minimum(x -> min(red(x), green(x), blue(x)), im)
    ub = maximum(x -> max(red(x), green(x), blue(x)), im)
    (im .- RGB(lb, lb, lb)) ./ (ub - lb)
end

In [ ]:
background_samples = sample(background, samples, H)

In [ ]:
seek(video, 4.0)
buf = read(video)

trace = hcat(map(1:40) do i
        for i in 1:6
            read!(video, buf)
        end
        pixels = sample(buf, samples, H)
        pixels .-= background_samples
        y = reshape(pixels, :)
        Lvx =  gain *  y
        Lv = Lvx[1]
        x = Lvx[2:end]
        x
        end...)

In [ ]:
imnormal(trace)